# Feature Engineering 

**Author:** Florian Klaver  

In this Jupyter Notebook the features for the modelling are created. This also contains the calculation of vegetation indices and sampling of the training data.

---
## Setup


In [1]:
import pandas as pd
import numpy as np
import rasterio
from scipy.ndimage import binary_erosion
import os
import glob
from tqdm.notebook import tqdm
from datetime import timedelta

In [2]:
# Paths 
sentinel_dir = '../data/Sentinel_CH/'
masks_dir = '../data/ground_truth_masks/'
features_output_dir = '../data/features_all_indices/' 

final_samples_output_path = '../data/training_samples_all_indices.csv'
cloud_stats_output_path = '../data/cloud_masking_stats.csv'

os.makedirs(features_output_dir, exist_ok=True)

In [3]:
# Parameters 
CLOUD_BAND = 13
CLOUD_THRESHOLD = 30  # Percent
N_POSITIVE_SAMPLES = 50000  # Mowed
N_NEGATIVE_SAMPLES = 50000  # Not mowed

# Time windows for temporal matching

# BEFORE_FAR: 9-20 days before mowing. This image will be used for the negative samples (not mowed)
# BEFORE_NEAR: 3-8 days before mowing
# AFTER: 1-7 days after mowing (at least 1 because we don't know the exact time of the mowing event).
# BEFORE_FAR and BEFORE_NEAR will be used to create negative samples (not mowed)
# BEFORE_NEAR and AFTER will be used to create positive samples (mowed)

BEFORE_FAR_MIN = 9
BEFORE_FAR_MAX = 20
BEFORE_NEAR_MIN = 3
BEFORE_NEAR_MAX = 8
AFTER_MIN = 1
AFTER_MAX = 7

# Feature List
FEATURE_NAMES = [
    # Indices AFTER (State) - 5 features
    'ndvi_after', 'evi_after', 'savi_after', 'gndvi_after', 'ndii_after',
    
    # Indices DIFF (Change) - 5 features
    'ndvi_diff', 'evi_diff', 'savi_diff', 'gndvi_diff', 'ndii_diff',
    
    # Raw Band DIFFs (Change) - 5 features
    'blue_diff', 'green_diff', 'red_diff', 'nir_diff', 'swir_diff',

    # Raw Bands AFTER (Optional state) - 5 features
    'blue_after', 'green_after', 'red_after', 'nir_after', 'swir_after'
]

Note: Other time windows and different approaches for searching the image date combinations than the chosen time windows were also tested. For example just choosing the two images before the mowing event (without time restrictions), a combination of both (using time constraints only for AFTER and BFORE_FAR) and other day limits for the given parameters. This chosen combination proofed to work the best in terms of matches found and data quality which directly influenced model performance.

---
### Temporal matching of Sentinel-2 and Ground Truth data

In [4]:
# Load Ground truth
masks_overview = pd.read_csv('../data/ground_truth_masks/masks_overview.csv')
masks_overview['date'] = pd.to_datetime(masks_overview['date'])


# Load Sentinel-2 files
sentinel_files = sorted(glob.glob(os.path.join(sentinel_dir, '*.tif')))
sentinel_dates = []

for f in sentinel_files:
    filename = os.path.basename(f).replace('.tif', '')
    try:
        date = pd.to_datetime(filename)
        sentinel_dates.append({'file': f, 'date': date, 'filename': filename})
    except Exception as e:
        print(f"Warning: Cannot parse date: {filename}. Error: {e}")

sentinel_df = pd.DataFrame(sentinel_dates)
print(f"Loaded {len(masks_overview)} ground truth records and {len(sentinel_df)} Sentinel-2 images.")


# Initialize list to store matches
matches = []

# For each ground truth mask, find matching Sentinel-2 images
for idx, row in tqdm(masks_overview.iterrows(), total=len(masks_overview), desc="Finding Matches"):
    event_date = row['date']
   

    after_window = sentinel_df[
        (sentinel_df['date'] >= event_date + timedelta(days=AFTER_MIN)) &
        (sentinel_df['date'] <= event_date + timedelta(days=AFTER_MAX))
    ]

    before_near_window = sentinel_df[
        (sentinel_df['date'] >= event_date - timedelta(days=BEFORE_NEAR_MAX)) &
        (sentinel_df['date'] <= event_date - timedelta(days=BEFORE_NEAR_MIN))
    ]

    before_far_window = sentinel_df[
        (sentinel_df['date'] >= event_date - timedelta(days=BEFORE_FAR_MAX)) &
        (sentinel_df['date'] <= event_date - timedelta(days=BEFORE_FAR_MIN))
    ]

   # If there is at least one image in each window, take the closest ones
    if len(after_window) > 0 and len(before_near_window) > 0 and len(before_far_window) > 0:
        img_after = after_window.iloc[0]
        img_before_near = before_near_window.iloc[-1]
        img_before_far = before_far_window.iloc[-1]

        matches.append({
            'event_date': event_date,
            'event_date_str': row['date_str'],
            'mask_file': row['filename'],
            'before_far_file': img_before_far['filename'] + '.tif',
            'before_near_file': img_before_near['filename'] + '.tif',
            'after_file': img_after['filename'] + '.tif',

        })


# Save matches to DataFrame for further processing
matches_df = pd.DataFrame(matches)
print(f"Found {len(matches_df)} matches with 3 valid images.")

# Save matches dataframe to csv for model assessment Pipeline
matches_df.to_csv('../data/temporal_matches.csv', index=False)


Loaded 227 ground truth records and 201 Sentinel-2 images.


Finding Matches:   0%|          | 0/227 [00:00<?, ?it/s]

Found 92 matches with 3 valid images.


In [5]:
display(matches_df)

,event_date,event_date_str,mask_file,before_far_file,before_near_file,after_file
0,2019-05-09,20190509,mask_20190509.tif,2019-04-30.tif,2019-05-05.tif,2019-05-10.tif
1,2019-05-16,20190516,mask_20190516.tif,2019-05-07.tif,2019-05-12.tif,2019-05-17.tif
2,2019-05-23,20190523,mask_20190523.tif,2019-05-12.tif,2019-05-20.tif,2019-05-25.tif
3,2019-05-30,20190530,mask_20190530.tif,2019-05-20.tif,2019-05-27.tif,2019-06-01.tif
4,2019-05-31,20190531,mask_20190531.tif,2019-05-20.tif,2019-05-27.tif,2019-06-01.tif
...,...,...,...,...,...,...
87,2023-09-05,20230905,mask_20230905.tif,2023-08-24.tif,2023-09-01.tif,2023-09-06.tif
88,2023-09-06,20230906,mask_20230906.tif,2023-08-24.tif,2023-09-01.tif,2023-09-08.tif
89,2023-09-07,20230907,mask_20230907.tif,2023-08-24.tif,2023-09-01.tif,2023-09-08.tif
90,2023-09-14,20230914,mask_20230914.tif,2023-09-01.tif,2023-09-11.tif,2023-09-16.tif


### Calculate relevant indices and Cloud masking

In this section the necessary vegetation indices (used as features for the model) are calculated and pixels with high cloud probability receive NAN-values so they don't generate wrong values and interfere with the models learning.

In [6]:
# All functions to calculate relevant vegetation indices

def calculate_ndvi(nir, red):
    """Calculates NDVI, handle division by zero"""
    with np.errstate(divide='ignore', invalid='ignore'):
        ndvi = (nir - red) / (nir + red)
    ndvi[np.isinf(ndvi)] = np.nan 
    return ndvi

def calculate_evi(nir, red, blue):
    """Enhanced Vegetation Index"""
    with np.errstate(divide='ignore', invalid='ignore'):
        evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))
    evi[np.isinf(evi)] = np.nan 
    return evi

def calculate_savi(nir, red, L=0.5):
    """Soil Adjusted Vegetation Index (L=0.5 is standard)"""
    with np.errstate(divide='ignore', invalid='ignore'):
        savi = ((nir - red) / (nir + red + L)) * (1 + L)
    savi[np.isinf(savi)] = np.nan
    return savi

def calculate_gndvi(nir, green):
    """Green Normalized Difference Vegetation Index"""
    with np.errstate(divide='ignore', invalid='ignore'):
        gndvi = (nir - green) / (nir + green)
    gndvi[np.isinf(gndvi)] = np.nan
    return gndvi

def calculate_ndii(nir, swir):
    """Normalized Difference Infrared Index (for water content)"""
    with np.errstate(divide='ignore', invalid='ignore'):
        ndii = (nir - swir) / (nir + swir)
    ndii[np.isinf(ndii)] = np.nan
    return ndii

In [7]:
# Initialize list to store cloud statistics
cloud_stats = []

# For each match, create feature rasters
for idx, row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Creating Feature Rasters"):
    
    # Relevant paths
    before_far_path = os.path.join(sentinel_dir, row['before_far_file'])
    before_near_path = os.path.join(sentinel_dir, row['before_near_file'])
    after_path = os.path.join(sentinel_dir, row['after_file'])
    
    try:
        with rasterio.open(before_far_path) as src_far, \
             rasterio.open(before_near_path) as src_near, \
             rasterio.open(after_path) as src_after:
            
            meta = src_near.meta.copy()
            
            # Read all bands
            # 'Far' bands (t-2)
            blue_far = src_far.read(1).astype(float)
            green_far = src_far.read(2).astype(float)
            red_far = src_far.read(3).astype(float)
            nir_far = src_far.read(7).astype(float)
            swir_far = src_far.read(10).astype(float)
            
            # 'Near' bands (t-1)
            blue_near = src_near.read(1).astype(float)
            green_near = src_near.read(2).astype(float)
            red_near = src_near.read(3).astype(float)
            nir_near = src_near.read(7).astype(float)
            swir_near = src_near.read(10).astype(float)
            
            # 'After' bands (t) 
            blue_after = src_after.read(1).astype(float)
            green_after = src_after.read(2).astype(float)
            red_after = src_after.read(3).astype(float)
            nir_after = src_after.read(7).astype(float)
            swir_after = src_after.read(10).astype(float)
            
            # Read cloud masks
            cloud_prob_far = src_far.read(CLOUD_BAND).astype(float)
            cloud_prob_near = src_near.read(CLOUD_BAND).astype(float)
            cloud_prob_after = src_after.read(CLOUD_BAND).astype(float)

    except Exception as e:
        print(f"Skipping match {row['event_date_str']} due to error loading files: {e}")
        continue

    # Process POSITIVE (Mowing) Pair (Near vs. After)
    cloud_mask_pos = (cloud_prob_near > CLOUD_THRESHOLD) | (cloud_prob_after > CLOUD_THRESHOLD)
    
    # Calculate 'before' (near) indices
    ndvi_before_pos = calculate_ndvi(nir_near, red_near)
    evi_before_pos = calculate_evi(nir_near, red_near, blue_near)
    savi_before_pos = calculate_savi(nir_near, red_near)
    gndvi_before_pos = calculate_gndvi(nir_near, green_near)
    ndii_before_pos = calculate_ndii(nir_near, swir_near)
    
    # Calculate 'after' indices
    ndvi_after_pos = calculate_ndvi(nir_after, red_after)
    evi_after_pos = calculate_evi(nir_after, red_after, blue_after)
    savi_after_pos = calculate_savi(nir_after, red_after)
    gndvi_after_pos = calculate_gndvi(nir_after, green_after)
    ndii_after_pos = calculate_ndii(nir_after, swir_after)
    
    # Stack features
    features_positive = np.stack([
        # Indices After
        ndvi_after_pos, evi_after_pos, savi_after_pos, gndvi_after_pos, ndii_after_pos,
        # Indices Diff
        ndvi_after_pos - ndvi_before_pos, 
        evi_after_pos - evi_before_pos, 
        savi_after_pos - savi_before_pos, 
        gndvi_after_pos - gndvi_before_pos, 
        ndii_after_pos - ndii_before_pos,
        # Band Diffs
        blue_after - blue_near, green_after - green_near, red_after - red_near, nir_after - nir_near, swir_after - swir_near,
        # Bands After
        blue_after, green_after, red_after, nir_after, swir_after
    ])
    # Apply cloud mask
    features_positive[:, cloud_mask_pos] = np.nan
    
    # Save POSITIVE features
    output_path_pos = os.path.join(features_output_dir, f'features_POSITIVE_{row["event_date_str"]}.tif')
    meta.update({'count': features_positive.shape[0], 'dtype': 'float32', 'nodata': np.nan})
    with rasterio.open(output_path_pos, 'w', **meta) as dst:
        dst.write(features_positive.astype('float32'))

    # Process NEGATIVE (No Mowing) Pair (Far vs. Near)
    cloud_mask_neg = (cloud_prob_far > CLOUD_THRESHOLD) | (cloud_prob_near > CLOUD_THRESHOLD)
    
    # Calculate 'before' (far) indices
    ndvi_before_neg = calculate_ndvi(nir_far, red_far)
    evi_before_neg = calculate_evi(nir_far, red_far, blue_far)
    savi_before_neg = calculate_savi(nir_far, red_far)
    gndvi_before_neg = calculate_gndvi(nir_far, green_far)
    ndii_before_neg = calculate_ndii(nir_far, swir_far)
    
    # Calculate 'after' (near) indices
    ndvi_after_neg = calculate_ndvi(nir_near, red_near)
    evi_after_neg = calculate_evi(nir_near, red_near, blue_near)
    savi_after_neg = calculate_savi(nir_near, red_near)
    gndvi_after_neg = calculate_gndvi(nir_near, green_near)
    ndii_after_neg = calculate_ndii(nir_near, swir_near)
    
    # Stack features
    features_negative = np.stack([
        # Indices After
        ndvi_after_neg, evi_after_neg, savi_after_neg, gndvi_after_neg, ndii_after_neg,
        # Indices Diff
        ndvi_after_neg - ndvi_before_neg, 
        evi_after_neg - evi_before_neg, 
        savi_after_neg - savi_before_neg, 
        gndvi_after_neg - gndvi_before_neg, 
        ndii_after_neg - ndii_before_neg,
        # Band Diffs
        blue_near - blue_far, green_near - green_far, red_near - red_far, nir_near - nir_far, swir_near - swir_far,
        # Bands After
        blue_near, green_near, red_near, nir_near, swir_near
    ])
    # Apply cloud mask
    features_negative[:, cloud_mask_neg] = np.nan
    
    # Save NEGATIVE features
    output_path_neg = os.path.join(features_output_dir, f'features_NEGATIVE_{row["event_date_str"]}.tif')
    meta.update({'count': features_negative.shape[0], 'dtype': 'float32', 'nodata': np.nan})
    with rasterio.open(output_path_neg, 'w', **meta) as dst:
        dst.write(features_negative.astype('float32'))

    # Statistics
    n_total = cloud_mask_pos.size
    n_cloudy_pos = np.sum(cloud_mask_pos)
    pct_cloudy_pos = (n_cloudy_pos / n_total) * 100
    n_cloudy_neg = np.sum(cloud_mask_neg)
    pct_cloudy_neg = (n_cloudy_neg / n_total) * 100
    
    # Store cloud stats
    cloud_stats.append({
        'event_date_str': row['event_date_str'],
        'pct_cloudy_pos': pct_cloudy_pos,
        'pct_cloudy_neg': pct_cloudy_neg
    })

Creating Feature Rasters:   0%|          | 0/92 [00:00<?, ?it/s]

In [8]:
cloud_stats_df = pd.DataFrame(cloud_stats)
cloud_stats_df.to_csv(cloud_stats_output_path, index=False)

---
### Balanced Sampling
Here points are sampled from the full set of negative and positive features. It is made sure to only sample pixels which lie below the set cloud probability threshold of 30% (= not NAN). Additionally a buffer of 10 meters (which equals 1 pixel) is introduced spanning the edges in all directions to account for the spatial shift (inaccuracy) given by the sentinel-2 imagery. By using this feature it is made sure to only sample more in the center of the mowed areas, lowering the chance of getting still faulty pixels (of streets, buildings or generally outside the actually mowed area) caused by the spatial shift.

In [9]:
# Calculate samples per match
samples_per_match_positive = max(1, N_POSITIVE_SAMPLES // len(matches_df))
samples_per_match_negative = max(1, N_NEGATIVE_SAMPLES // len(matches_df))

print(f"Targeting {N_POSITIVE_SAMPLES} positive and {N_NEGATIVE_SAMPLES} negative samples.")
print(f"  → ~{samples_per_match_positive} positive samples per match (from mask==1)")
print(f"  → ~{samples_per_match_negative} negative samples per match (from mask==1)\n")

# Initialize list to store all samples
all_samples = []

# For each match, sample positive and negative samples
for idx, match_row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Sampling (Pos/Neg)"):
    
    event_date = pd.to_datetime(match_row['event_date'])
    
    features_pos_path = os.path.join(features_output_dir, f'features_POSITIVE_{match_row["event_date_str"]}.tif')
    features_neg_path = os.path.join(features_output_dir, f'features_NEGATIVE_{match_row["event_date_str"]}.tif')
    mask_path = os.path.join(masks_dir, match_row['mask_file'])
    
    try:
        with rasterio.open(features_pos_path) as feat_pos_src, \
             rasterio.open(features_neg_path) as feat_neg_src, \
             rasterio.open(mask_path) as mask_src:
            
            # Read the data
            features_pos = feat_pos_src.read()
            features_neg = feat_neg_src.read()
            mask = mask_src.read(1)
            
            # Apply Negative Buffer (Erosion)
            # --> Shrink the mask by 1 pixel (10m) to avoid edge effects/shift
            mask_eroded = binary_erosion(mask == 1, structure=np.ones((3,3)), iterations=1)
            
            # Use the ERODED mask as final polygon mask
            polygon_mask = (mask_eroded == 1)
            
            # Check if there are any pixels left after erosion
            if np.sum(polygon_mask) == 0:
                continue
            
            # Find Valid Pixels (Robust Sampling)
            # Positive: Inside Eroded Polygon AND Not NaN
            valid_mask_pos = polygon_mask & ~np.isnan(features_pos[0])
            valid_indices_pos = np.where(valid_mask_pos)
            n_valid_pos = len(valid_indices_pos[0])
            
            # Negative: Inside Eroded Polygon AND Not NaN
            valid_mask_neg = polygon_mask & ~np.isnan(features_neg[0])
            valid_indices_neg = np.where(valid_mask_neg)
            n_valid_neg = len(valid_indices_neg[0])

            # Extract Positive Samples
            if n_valid_pos > 0:
                n_to_sample = min(samples_per_match_positive, n_valid_pos)
                choices = np.random.choice(n_valid_pos, size=n_to_sample, replace=False)
                
                rows = valid_indices_pos[0][choices]
                cols = valid_indices_pos[1][choices]
                
                for r, c in zip(rows, cols):
                    feature_values = features_pos[:, r, c]
                    x, y = feat_pos_src.xy(r, c)
                    all_samples.append({
                        'x': x, 'y': y, 'event_date': event_date,
                        'match_id': idx, 'label': 1,
                        **{FEATURE_NAMES[i]: feature_values[i] for i in range(len(FEATURE_NAMES))}
                    })

            # Extract Negative Samples
            if n_valid_neg > 0:
                n_to_sample = min(samples_per_match_negative, n_valid_neg)
                choices = np.random.choice(n_valid_neg, size=n_to_sample, replace=False)
                
                rows = valid_indices_neg[0][choices]
                cols = valid_indices_neg[1][choices]
                
                for r, c in zip(rows, cols):
                    feature_values = features_neg[:, r, c]
                    x, y = feat_neg_src.xy(r, c)
                    all_samples.append({
                        'x': x, 'y': y, 'event_date': event_date,
                        'match_id': idx, 'label': 0,
                        **{FEATURE_NAMES[i]: feature_values[i] for i in range(len(FEATURE_NAMES))}
                    })

    except Exception as e:
        print(f"\nSkipping match {match_row['event_date_str']} due to error: {e}")
        continue

# Save all samples to CSV
samples_df = pd.DataFrame(all_samples)

Targeting 50000 positive and 50000 negative samples.
  → ~543 positive samples per match (from mask==1)
  → ~543 negative samples per match (from mask==1)



Sampling (Pos/Neg):   0%|          | 0/92 [00:00<?, ?it/s]

In [10]:
print(f"Total Samples Generated: {len(samples_df):,}")
print(f"  Mowed (1): {(samples_df['label'] == 1).sum():,} ({(samples_df['label'] == 1).sum() / len(samples_df) * 100:.1f}%)")
print(f"  Not Mowed (0): {(samples_df['label'] == 0).sum():,} ({(samples_df['label'] == 0).sum() / len(samples_df) * 100:.1f}%)")

# Save final samples to CSV for model training
samples_df.to_csv(final_samples_output_path, index=False)
print(f"\nSaved final samples to: {final_samples_output_path}")

print("\nFirst 5 samples:")
display(samples_df.head())


Total Samples Generated: 33,368
  Mowed (1): 18,077 (54.2%)
  Not Mowed (0): 15,291 (45.8%)

Saved final samples to: ../data/training_samples_all_indices.csv

First 5 samples:


,x,y,event_date,match_id,label,ndvi_after,evi_after,savi_after,gndvi_after,ndii_after,...,blue_diff,green_diff,red_diff,nir_diff,swir_diff,blue_after,green_after,red_after,nir_after,swir_after
0,2.684419e+06,1.257664e+06,2019-06-23,12,1,0.605621,2.276109,0.908324,0.547755,0.084742,...,382.603241,298.437805,444.256653,-1931.239014,919.115601,745.605896,993.012207,834.746582,3398.470459,2867.478516
1,2.682788e+06,1.259725e+06,2019-06-23,12,1,0.551926,1.688797,0.827774,0.511190,0.033511,...,292.290039,258.544739,427.948334,-1470.512695,929.918579,622.583740,899.156921,802.588928,2779.806152,2599.537109
2,2.682948e+06,1.259115e+06,2019-06-23,12,1,0.523062,1.512825,0.784500,0.509137,0.029171,...,394.649963,365.286743,573.695435,-595.651367,1154.615845,742.769470,1038.889893,1000.189819,3194.023926,3012.962891
3,2.683208e+06,1.258165e+06,2019-06-23,12,1,0.542529,1.637401,0.813699,0.507698,0.078942,...,406.068481,350.973083,544.185181,-596.735107,1078.000000,756.089478,1085.528320,985.947571,3324.479248,2838.000000
4,2.683339e+06,1.257925e+06,2019-06-23,12,1,0.519052,1.472934,0.778488,0.497168,0.131705,...,386.119598,414.879395,526.435913,-396.569092,787.685547,760.185974,1100.451050,1037.392578,3276.558838,2513.921631


Slight Class imbalance still remains, but it is small enough to be neglected.